In [1]:
import os

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable
import torch.nn.init as weight_init

from sklearn.model_selection import train_test_split

In [2]:
MODEL_DIR = "Model"
MODEL_VER = "1"
MODEL_FILE = "model.pt"
MODEL_PATH = os.path.join(MODEL_DIR, MODEL_VER)

if not os.path.exists(MODEL_PATH):
    os.makedirs(MODEL_PATH)

# Model

In [3]:
lr = 0.01
dropout = 0.1
epochs = 200
activation = "sigmoid"

In [4]:
"""Reference: https://medium.com/@haoyunlai/pytorch-implementation-of-autoencoder-based-recommender-system-9aff6c3d1b02"""

class Encoder(nn.Module):
    """
    Class for Symmetrical AutoEncoder Network
    :param L: List of int, contains sizes of encoding layers and starts with feature size
    For example: [500, 20, 10] will result in:
      - encoder 2 layers: 500x20 and 20x10. Representation layer (z) will be 10
      - decoder 2 layers: 10x20 and 20x500, output size is 500, reconstructed.
    :param activation_fn: (default 'sigmoid') Type of activation function
    :param drop_prob: (default: 0.0) Dropout probability
    """
    def __init__(self, L, activation_fn='sigmoid', drop_prob=0.0):
        super(Encoder, self).__init__()
        layers = self.create_nn_structure(L)
        self.num_layers = len(L)
        # create controller for activation function
        self.activation_fn_nm = activation_fn
        # create dropout module
        self._drop_prob = drop_prob
        if drop_prob > 0.0:
            self.dropout = nn.Dropout(drop_prob)
        #initialize with empty list to store layers
        self.linears = nn.ModuleList([])
        self.linears.extend([nn.Linear(i[0], i[1]) for i in layers])
        
    def get_activation_fn(self):
        # user selected activation function at layers except for last layer
        if self.activation_fn_nm == 'relu':
            return nn.ReLU()
        elif self.activation_fn_nm == 'lrelu':
            return nn.LeakyReLU()
        elif self.activation_fn_nm == 'sigmoid':
            return nn.Sigmoid()
        else:
            raise ValueError('Activation function type not defined')
    
    def forward(self, x):
        for i,layer in enumerate(self.linears):
            if i <= self.num_layers-1:
                # create instance of activation function
                act_fn = self.get_activation_fn()
                # pass in the input
                x = act_fn(self.linears[i](x))
                if self._drop_prob > 0.0 and i <= int(self.num_layers/2): 
                    # apply dropout only on encode layer by control of i
                    x = self.dropout(x)
        # No activation on the last decoding layer
        x = self.linears[-1](x)
        return x

    def create_nn_structure(self, L):
        max_ind = len(L)-1
        layers = []
        for i,v in enumerate(L):
            if i < max_ind:
                #still have i+1 available, create layer tuple
                layer = [v,L[i+1]]
                layers.append(layer)
        #then inverse the layers for decoder size
        encoder_layers = layers[:]
        for l in encoder_layers[::-1]:
            decoder_layer = l[::-1]
            layers.append(decoder_layer)
        return layers

### Data Preprocessing

In [5]:
def create_index_mapping(L):
    '''
    return reindexed dict on user and items
    encoded indices starts from 1
    input: 
    * L: list of str
    outputs:
    * ind_2_item,item_2_ind: tuple of dictionary
    '''
    L = set(L)
    ind_2_item = {}
    
    for i,v in enumerate(L):
        #index start from 1
        ind_2_item[i+1] = v
    #invert the map
    item_2_ind = {v: k for k, v in ind_2_item.items()}
    return ind_2_item,item_2_ind

def reindexer(ratings_df,user_col,item_col,rating_col):
    '''
    inputs:
    * ratings_df: pandas df containing ratings/affinity for user-item pairs
    * user_col: actual col name for users
    * item_col: actual col name for items
    * rating_col: actual col name for ratings
    output:
    * ratings_df: reindexed user and item column, pandas df
    '''
    users_list = ratings_df[user_col].tolist()
    item_list = ratings_df[item_col].tolist()
    
    ind_2_user,user_2_ind = create_index_mapping(users_list)
    ind_2_item,item_2_ind = create_index_mapping(item_list)
    
    #rename ratings df
    ratings_df = ratings_df.rename(columns={user_col:'user_col',
                                            item_col:'item_col',
                                            rating_col:'rating_col'})
    
    #encode df using the 2 mappings
    ratings_df['encoded_users'] = ratings_df['user_col'].apply(lambda x:user_2_ind[x])
    ratings_df['encoded_items'] = ratings_df['item_col'].apply(lambda x:item_2_ind[x])
    
    return ratings_df[['encoded_users','encoded_items','rating_col']], ind_2_user,user_2_ind, ind_2_item,item_2_ind

def convert(data):
    new_data = []
    for id_users in range(nb_users+1):
        # each user's watched movies
        # data[:,0], first column, all rows column users
        id_items = data[:,1][data[:,0] == id_users]
        # each user's rating for that item
        id_ratings = data[:,2][data[:,0] == id_users]
        ratings = np.zeros(nb_movies)
        # the positions of these items are filled with ratings, creating the matrix
        ratings[id_items-1] = id_ratings
        new_data.append(list(ratings))
    return new_data

# Data loading and preprocessing

In [6]:
ratings = pd.read_csv('MovieLens1M/ratings.dat', delimiter = '::',header=None, engine='python')

ratings.columns = ['userid','movieid','rating','timestamp']
ratings_reindex, ind_2_user,user_2_ind , ind_2_item,item_2_ind = reindexer(ratings,'userid','movieid','rating')


train, test = train_test_split(ratings_reindex,
                               stratify=ratings_reindex['encoded_users'],
                               test_size=0.1,
                               random_state=42)

training_set = np.array(train, dtype='int')
test_set = np.array(test, dtype='int')

nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))
  
training_set = convert(training_set)
test_set = convert(test_set)

training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [7]:
print(test_set)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


# Training and Validation

In [8]:
autoencoder_network = Encoder([nb_movies, 20,10], activation, dropout)
criterion = nn.MSELoss()
optimizer = optim.RMSprop(autoencoder_network.parameters(), lr = lr, weight_decay = 0.5)

validationLoss = float("inf")
minEpoch = epochs
nb_epoch = epochs


for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    # s is the number of users who rated at least 1 movies
    for id_user in range(nb_users):
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = input.clone()
        if torch.sum(target.data > 0) > 0:
            output = autoencoder_network(input)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10) #making this anyway not equal to 0, as this will be a denominator
            #mean_corrector is the avg of the error, only considering the movies having ratings (non-zero ratings) for computing mean of error
            loss.backward() # decide the direction the increment of weights
            #this call will just computing all the gradients required
            train_loss += np.sqrt(loss.data*mean_corrector)
            s += 1.
            optimizer.step() # decide the amount to update the weights
    
    # Validation
    test_loss = 0
    s = 0.

    res = []
    targets = []

    # averaged difference between real rating and predicted rating
    for id_user in range(nb_users):
        input = Variable(training_set[id_user]).unsqueeze(0) # should keep the training set
        target = Variable(test_set[id_user]).unsqueeze(0) # to predict the other movies user not seen yet

        if torch.sum(target.data > 0) > 0:
            # make predictions
            with torch.no_grad():
                output = autoencoder_network(input)
                targets.append(target.detach().numpy())
                res.append(output.detach().numpy()) 
                target.require_grad = False
                output[target == 0] = 0 # dont want to measure the loss on the movies didnt get the actual rating from user 
                # force to 0 and difference / loss will be 0 for those entries
                loss = criterion(output, target)

                mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10) 
                # only consider the movies that are rated in the test set, to be included in the loss
                test_loss += np.sqrt(loss.data*mean_corrector)
                s += 1.
    print(f"epoch: {epoch} train loss: {train_loss/s}|| test loss: {test_loss/s}")
    
    # Saving Model
    if validationLoss > (test_loss/s):
        validationLoss = (test_loss/s)
        minEpoch = epoch
        torch.save(autoencoder_network.state_dict(), os.path.join(MODEL_PATH,MODEL_FILE))

epoch: 1 train loss: 1.279646873474121|| test loss: 0.987733781337738
epoch: 2 train loss: 0.9990950226783752|| test loss: 0.9645319581031799
epoch: 3 train loss: 0.9853835105895996|| test loss: 0.9585108160972595
epoch: 4 train loss: 0.9803895950317383|| test loss: 0.9558501839637756
epoch: 5 train loss: 0.9784775972366333|| test loss: 0.9556391835212708
epoch: 6 train loss: 0.9769519567489624|| test loss: 0.9541357755661011
epoch: 7 train loss: 0.9765042662620544|| test loss: 0.9539057612419128
epoch: 8 train loss: 0.9758345484733582|| test loss: 0.9530025124549866
epoch: 9 train loss: 0.9755560159683228|| test loss: 0.9529211521148682
epoch: 10 train loss: 0.9750311970710754|| test loss: 0.9525985717773438
epoch: 11 train loss: 0.9752235412597656|| test loss: 0.9529465436935425
epoch: 12 train loss: 0.9746390581130981|| test loss: 0.9520230889320374
epoch: 13 train loss: 0.974666953086853|| test loss: 0.9521762728691101
epoch: 14 train loss: 0.9745326638221741|| test loss: 0.9520975

epoch: 114 train loss: 0.9442179799079895|| test loss: 0.925438642501831
epoch: 115 train loss: 0.943023681640625|| test loss: 0.9239884614944458
epoch: 116 train loss: 0.9423819780349731|| test loss: 0.9243716597557068
epoch: 117 train loss: 0.9407351613044739|| test loss: 0.9214136600494385
epoch: 118 train loss: 0.9398788809776306|| test loss: 0.9213270545005798
epoch: 119 train loss: 0.9407896399497986|| test loss: 0.9243089556694031
epoch: 120 train loss: 0.945716142654419|| test loss: 0.932594895362854
epoch: 121 train loss: 0.9511318802833557|| test loss: 0.9286588430404663
epoch: 122 train loss: 0.9481267929077148|| test loss: 0.9260352253913879
epoch: 123 train loss: 0.9460569024085999|| test loss: 0.9257525205612183
epoch: 124 train loss: 0.9472229480743408|| test loss: 0.9255449771881104
epoch: 125 train loss: 0.9454115629196167|| test loss: 0.9234623908996582
epoch: 126 train loss: 0.9430842995643616|| test loss: 0.9215109944343567
epoch: 127 train loss: 0.9432399272918701|

In [9]:
print(validationLoss, epoch)

tensor(0.9004) 200


# Making top k recommendation

In [10]:
def make_top_k_recommendations(encoder,evidence,k,filter_seen=True):
    '''
    :param encoder: autoencoder instance
    :param evidence: full set of seen ratings from all users
    :param k: top k items (by output score)
    :param filter_seen: (default True) filter controller to remove seen items from top k list
    '''     
    res = []
    nb_users = evidence.shape[0]
    # to find top scored items for each user
    for id_user in range(nb_users):
        encoder_input = Variable(evidence[id_user]).unsqueeze(0) # should keep the training set 
        encoder_output = encoder(encoder_input)
        
        target = Variable(evidence[id_user]).unsqueeze(0) # mask to find items not seen yet
        if filter_seen:
            encoder_output[target != 0] = 0 # force seen items scores to 0, will never get recommended
        res.append(encoder_output.detach().numpy())
        
    res = [a[0] for a in res]
    final_itemsets = []    
    for each in res:
        full_ratings_predicted = list(each)
        full_ratings_indexed = list(enumerate(full_ratings_predicted))
        final_itemsets.append(sorted(full_ratings_indexed,key=lambda x:x[1],reverse =True)[:k])
        
    return final_itemsets